# EXPLORATORY DATA ANALYSIS

## Packages and Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

2024-11-12 22:01:01.492805: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 22:01:01.637434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 22:01:01.722735: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 22:01:01.753189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 22:01:01.897107: I tensorflow/core/platform/cpu_feature_guar

## Data Analysis and Preprocessing

In [2]:
# Train and test data from Tox21 dataset
x_train = pd.read_csv('data/tox21_dense_train.csv')
y_train = pd.read_csv('data/tox21_labels_train.csv')

x_test = pd.read_csv('data/tox21_dense_test.csv')
y_test = pd.read_csv('data/tox21_labels_test.csv')

In [3]:
df_train = pd.merge(x_train, y_train, on='Unnamed: 0', how='inner')
df_train.head()

,Unnamed: 0,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,...,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
0,NCGC00178831-03,5.436720e+07,13.053,2.176,3.194,23.112,15.868,1.496,15.127,12.592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,NCGC00166114-03,1.268818e+07,22.123,2.065,3.137,21.033,13.718,1.937,13.187,11.951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,NCGC00263563-01,3.076932e+06,13.085,2.154,3.207,46.896,29.958,3.806,30.105,25.569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,NCGC00013058-02,7.168569e+07,12.832,2.029,3.380,51.086,32.045,1.806,29.090,21.603,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,NCGC00167516-01,7.989702e+06,12.936,2.124,3.573,70.295,46.402,3.604,42.132,32.570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print('Train data shape:', x_train.shape)
print('Train labels shape:', y_train.shape)
print('Overall train data shape:', df_train.shape)

Train data shape: (12060, 802)
Train labels shape: (12060, 13)
Overall train data shape: (12060, 814)


In [5]:
from sklearn.model_selection import train_test_split
df_cv, df_final_train = train_test_split(df_train, test_size=0.8, random_state=42)

In [6]:
print("=== PRE-FILTERING SHAPE ===")
print('Cross-validation data shape:', df_cv.shape)
print('Final train data shape:', df_final_train.shape)

=== PRE-FILTERING SHAPE ===
Cross-validation data shape: (2412, 814)
Final train data shape: (9648, 814)


In [7]:
# Calculate label density for each compound
df_cv['label_density'] = df_cv.iloc[:, -12:].notna().sum(axis=1)

# now we keep only compounds with at least 8 labels in the cross validation set
df_cv_filtered = df_cv[df_cv['label_density'] >= 8]

# move the rest to the final training set
df_final_train = pd.concat([df_final_train, df_cv[df_cv['label_density'] < 8]])

In [8]:
print("=== POST-FILTERING SHAPE ===")
print('Cross-validation data shape:', df_cv_filtered.shape)
print('Final train data shape:', df_final_train.shape)

=== POST-FILTERING SHAPE ===
Cross-validation data shape: (1732, 815)
Final train data shape: (10328, 815)


## Model definition and Training Logic

#### MultiTask Model Definition

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# Define model training code
class MultiTaskToxicityModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_tasks=12, dropout_rate=0.5):
        super(MultiTaskToxicityModel, self).__init__()

        # Hidden layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)

        # dropout for regularization
        self.dropout = nn.Dropout(dropout_rate)

        # output layers, and we have one for each task
        self.output_layers = nn.ModuleList([nn.Linear(hidden_dim, 1) for _ in range(num_tasks)])
    
    def forward(self, x):
        # pass through hidden layers with relu activations and dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)

        # pass through output layers
        outputs = [torch.sigmoid(output_layer(x)) for output_layer in self.output_layers]

        # Concatenate outputs for each task along the batch dimension
        return torch.cat(outputs, dim=1)

#### Training Objective

In [10]:
####
def masked_bce_loss(y_pred, y_true, mask):
    # clamp predictions to avoid log(0)
    eps = 1e-7
    y_pred = torch.clamp(y_pred, eps, 1 - eps)

    # compute individual binary crossentropy loss
    bce_loss = - (y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))
    
    # apply the mask
    masked_bce_loss = bce_loss * mask

    # return the mean loss
    return masked_bce_loss.sum()



#### Model Training Logic

In [11]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

def train_model(model, optimizer, X_train, y_train, epochs, batch_size):
    # Convert data to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

    # Create a binary mask (1 for valid labels, 0 for NaNs in y_train)
    mask_tensor = torch.isnan(y_train_tensor).logical_not().float()

    # Replace NaNs in y_train_tensor with zeros (they won't contribute to loss due to masking)
    y_train_tensor[torch.isnan(y_train_tensor)] = 0

    # Create a TensorDataset and DataLoader
    train_data = TensorDataset(X_train_tensor, y_train_tensor, mask_tensor)
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

    # Set model to training mode
    model.train()

    # Begin training loop
    for epoch in range(epochs):
        total_loss = 0  # Accumulate loss over all batches in this epoch

        for i, (X_batch, y_batch, mask_batch) in enumerate(train_loader):
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = model(X_batch)

            # Calculate masked binary cross-entropy loss
            loss = masked_bce_loss(y_pred, y_batch, mask_batch)  # Pass mask_batch, not mask_tensor
            total_loss += loss.item()  # Accumulate batch loss

            # Backward pass
            loss.backward()
            optimizer.step()

        # Print average loss for the epoch
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss}")

    # Return the trained model
    return model


## Start Training And Initialize Training Config

In [12]:
learning_rate = 1e-4
epochs = 10
batch_size = 32
num_tasks = 12

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

kf = KFold(n_splits=5, random_state=42, shuffle=True)

imputer = SimpleImputer(strategy='median')

cv_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(df_cv_filtered)):
    fold_train_set = df_cv_filtered.iloc[train_idx]
    fold_val_set = df_cv_filtered.iloc[val_idx]

    # combine fold train set with final train set
    combined_train_set = pd.concat([df_final_train, fold_train_set], axis=0)

    # separate features and labels
    X_train = combined_train_set.iloc[:, 1:-12]
    y_train = combined_train_set.iloc[:, -13:]
    X_val = fold_val_set.iloc[:, 1:-12]
    y_val = fold_val_set.iloc[:, -13:]

    # Impute feature vectors via median imputation
    X_train = imputer.fit_transform(X_train)
    X_val = imputer.transform(X_val)

    # standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train) # this converts to numpy array
    X_val = scaler.transform(X_val) # this converts to numpy array

    # Drop last column of y_train and y_val which is the label_density
    y_train = y_train.iloc[:, :-1]
    y_val = y_val.iloc[:, :-1].fillna(y_val.median())

    # Reset indices for alignment
    y_val.reset_index(drop=True, inplace=True)

    # initialize model
    model = MultiTaskToxicityModel(input_dim=X_train.shape[1])

    # initialize optimizer
    optimizer = Adam(model.parameters(), lr=0.001)

    # initialize loss function
    loss_fn = nn.BCELoss(reduction='none')

    # train model
    model = train_model(model, optimizer, X_train, y_train, epochs, batch_size)

    fold_results = {}
    
    for task_idx, task in enumerate(y_train.columns):  # Use task index for slicing y_pred_task
        # Get the target values for the current task
        y_val_task = y_val[task]

        # Align features (X_val) with the corresponding indices in y_val_task
        X_val_task = X_val[y_val_task.index.to_numpy(), :]  # Use NumPy-style indexing

        # Predict using the model
        y_pred_task = model(torch.tensor(X_val_task, dtype=torch.float32)).detach().numpy()

        # Extract predictions for the current task (task_idx)
        y_pred_task_specific = y_pred_task[:, task_idx]  # Select the column corresponding to the task

        # Compute AUC for the current task
        auc = roc_auc_score(y_val_task, y_pred_task_specific)
        fold_results[task] = auc

    
    # Store results for this fold
    cv_results.append(fold_results)
    print(f"Fold {fold + 1} AUCs: {fold_results}")


Epoch 1/10, Loss: 59.99110905545934
Epoch 2/10, Loss: 51.54362818428216
Epoch 3/10, Loss: 50.240195173658535
Epoch 4/10, Loss: 49.079479768425635
Epoch 5/10, Loss: 48.40959407068403
Epoch 6/10, Loss: 47.42725778925321
Epoch 7/10, Loss: 47.38398640158736
Epoch 8/10, Loss: 46.844275036689695
Epoch 9/10, Loss: 46.37939709221959
Epoch 10/10, Loss: 45.82382053266754
Fold 1 AUCs: {'NR.AhR': 0.9998168498168498, 'NR.AR': 0.9129129129129129, 'NR.AR.LBD': 0.8691394658753709, 'NR.Aromatase': 0.7383300460223536, 'NR.ER': 0.7600979192166463, 'NR.ER.LBD': 0.9306164183464796, 'NR.PPAR.gamma': 0.8042016806722688, 'SR.ARE': 0.7404761904761905, 'SR.ATAD5': 0.8012436665131276, 'SR.HSE': 0.6270029673590505, 'SR.MMP': 0.8214285714285714, 'SR.p53': 0.791566265060241}
Epoch 1/10, Loss: 60.60493838137437
Epoch 2/10, Loss: 52.124519833335675
Epoch 3/10, Loss: 50.38657640647499
Epoch 4/10, Loss: 49.222499958851074
Epoch 5/10, Loss: 47.90199815106327
Epoch 6/10, Loss: 47.32719528423343
Epoch 7/10, Loss: 46.94420

In [15]:
# convert results to a DataFrame for easier analysis
cv_results_df = pd.DataFrame(cv_results)

# calculate mean auc for each task across all folds
mean_aucs = cv_results_df.mean(axis=0)

print('Mean AUCs for each task:', mean_aucs)

Mean AUCs for each task: NR.AhR           0.999239
NR.AR            0.824792
NR.AR.LBD        0.856683
NR.Aromatase     0.846900
NR.ER            0.714453
NR.ER.LBD        0.870623
NR.PPAR.gamma    0.838962
SR.ARE           0.754884
SR.ATAD5         0.822241
SR.HSE           0.727214
SR.MMP           0.833390
SR.p53           0.814735
dtype: float64


## Hyperparameter Tuning

In [16]:
best_epochs = 10
best_batch_size = 32
best_learning_rate = 0.05

## Final Training Round

In [17]:
# Combine the entire dataset for final training
X_final_train = df_final_train.iloc[:, 1:-12]
y_final_train = df_final_train.iloc[:, -12:]

print(X_final_train.shape)
print(y_final_train.shape)

model = MultiTaskToxicityModel(input_dim=X_final_train.shape[1])
optimizer = Adam(model.parameters(), lr=best_learning_rate)

# scale features
X_final_train = scaler.fit_transform(X_final_train)

# impute missing values
X_final_train = imputer.fit_transform(X_final_train)

# Train final model with optimized parameters
final_model = train_model(model, optimizer, X_final_train, y_final_train, best_epochs, best_batch_size)


(10328, 802)
(10328, 12)


Epoch 1/10, Loss: 199.1561598910636
Epoch 2/10, Loss: 199.2553668125495
Epoch 3/10, Loss: 199.25536887343085
Epoch 4/10, Loss: 199.25536785775293
Epoch 5/10, Loss: 199.2553679965229
Epoch 6/10, Loss: 199.25536783154905
Epoch 7/10, Loss: 199.25536805852647
Epoch 8/10, Loss: 199.25536815300813
Epoch 9/10, Loss: 199.2553675270671
Epoch 10/10, Loss: 199.25536843645315


## Testing 

## Metric Report